In [1]:
import cv2
import face_recognition
import firebase_admin
from firebase_admin import credentials, firestore
import requests
import numpy as np
from datetime import datetime
import os
from io import BytesIO
from PIL import Image
#0000000
# --- Configurations ---
cred = credentials.Certificate('firebase_credentials.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

COLLECTION_NAME = "students"
ATTENDANCE_DIR = "attendance_logs"
TOLERANCE = 0.6

if not os.path.exists(ATTENDANCE_DIR):
    os.makedirs(ATTENDANCE_DIR)

timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
attendance_file = os.path.join(ATTENDANCE_DIR, f"Attendance_{timestamp_str}.csv")


if not os.path.exists(attendance_file):
    with open(attendance_file, 'w') as f:
        f.write("Name,Regd No.,Time\n")

# --- Initialize Firebase Attendance Document ---
session_id = datetime.now().strftime('attendance_%Y%m%d_%H%M%S')
attendance_doc_ref = db.collection('attendance').document(session_id)
attendance_data = {}
student_counter = 0
already_marked = set()

# --- Load Faces from Firebase ---
print("🔄 Fetching face data from Firebase...")
students_ref = db.collection(COLLECTION_NAME)
docs = students_ref.stream()

known_face_encodings = []
known_face_names = []
known_face_regnos = []

for doc in docs:
    data = doc.to_dict()
    name = data.get('name')
    reg_no = data.get('reg_no')
    photo_url = data.get('photo_url')

    try:
        response = requests.get(photo_url)
        img = Image.open(BytesIO(response.content)).convert('RGB')
        img_np = np.array(img)

        encodings = face_recognition.face_encodings(img_np)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(name)
            known_face_regnos.append(reg_no)
            print(f"✅ Loaded: {name} ({reg_no})")
        else:
            print(f"❌ Face not found in image: {name}")

    except Exception as e:
        print(f"⚠️ Error loading {name}: {e}")

print("✅ Face data loading complete.\n")

# --- Mark Attendance Function ---
def mark_attendance(name, reg_no):
    global student_counter

    if reg_no in already_marked:
        return

    now = datetime.now()
    timestamp = now.strftime('%I:%M %p')  # e.g., 12:05 PM

    # Write to CSV
    with open(attendance_file, 'a') as f:
        f.write(f"{name},{reg_no},{now.strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Add to Firebase batch data
    student_key = f"student_{student_counter}"
    attendance_data[student_key] = {
        'name': name,
        'reg_no': reg_no,
        'time': timestamp
    }

    already_marked.add(reg_no)
    student_counter += 1

    print(f"🟢 Attendance marked for: {name} ({reg_no}) at {timestamp}")

# --- Start Webcam & Recognize Faces ---
print("🎥 Starting webcam. Press 'q' to quit.")
cap = cv2.VideoCapture(0)

#url = 'http://192.0.0.4:8080/video'  # Your IP webcam stream URL
#cap = cv2.VideoCapture(url)


if not cap.isOpened():
    print("❌ Failed to access webcam.")
    exit()

try:
    while True:
        success, frame = cap.read()
        if not success:
            break

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=TOLERANCE)
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            name = "Unknown"
            reg_no = ""

            if face_distances.size > 0:
                best_match = np.argmin(face_distances)
                if matches[best_match]:
                    name = known_face_names[best_match]
                    reg_no = known_face_regnos[best_match]
                    mark_attendance(name, reg_no)

            # Draw bounding box
            top, right, bottom, left = [v * 4 for v in face_location]
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            label = f"{name} ({reg_no})" if name != "Unknown" else name
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, label, (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 1)

        cv2.imshow('Face Recognition Attendance', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("⛔ Interrupted by user.")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("👋 Attendance session ended.")

    # --- Upload Attendance Data to Firebase ---
    if attendance_data:
        try:
            attendance_doc_ref.set(attendance_data)
            print(f"📤 Attendance session uploaded to Firebase as '{session_id}'")
        except Exception as e:
            print(f"❌ Error uploading attendance: {e}")


🔄 Fetching face data from Firebase...
✅ Loaded: Akash Kumar Tripathy (2101104060)
✅ Loaded:  Birajananda Tripathy (2101104071)
✅ Loaded: Jyotirmaya sethy (2101104080)
✅ Loaded: Sabyasachi Mallick  (2101104095)
✅ Loaded: Bhabani Sankar Samal (2221104002)
✅ Face data loading complete.

🎥 Starting webcam. Press 'q' to quit.
🟢 Attendance marked for: Akash Kumar Tripathy (2101104060) at 08:48 PM
🟢 Attendance marked for: Jyotirmaya sethy (2101104080) at 08:48 PM
👋 Attendance session ended.
📤 Attendance session uploaded to Firebase as 'attendance_20250427_204732'
